# Работа интерпретатора
## Шаг 1: Исходный код

In [2]:
def add(a, b):
    return a + b

result = add(3, 3)
print(result)

6


## Шаг 2: Лексический анализ — разбиение на токены

In [2]:
import tokenize
from io import BytesIO

code = """
def add(a, b):
    return a + b

result = add(2, 3)
print(result)
"""

tokens = tokenize.tokenize(BytesIO(code.encode('utf-8')).readline)
for token in tokens:
    print(token)

TokenInfo(type=62 (ENCODING), string='utf-8', start=(0, 0), end=(0, 0), line='')
TokenInfo(type=61 (NL), string='\n', start=(1, 0), end=(1, 1), line='\n')
TokenInfo(type=1 (NAME), string='def', start=(2, 0), end=(2, 3), line='def add(a, b):\n')
TokenInfo(type=1 (NAME), string='add', start=(2, 4), end=(2, 7), line='def add(a, b):\n')
TokenInfo(type=54 (OP), string='(', start=(2, 7), end=(2, 8), line='def add(a, b):\n')
TokenInfo(type=1 (NAME), string='a', start=(2, 8), end=(2, 9), line='def add(a, b):\n')
TokenInfo(type=54 (OP), string=',', start=(2, 9), end=(2, 10), line='def add(a, b):\n')
TokenInfo(type=1 (NAME), string='b', start=(2, 11), end=(2, 12), line='def add(a, b):\n')
TokenInfo(type=54 (OP), string=')', start=(2, 12), end=(2, 13), line='def add(a, b):\n')
TokenInfo(type=54 (OP), string=':', start=(2, 13), end=(2, 14), line='def add(a, b):\n')
TokenInfo(type=4 (NEWLINE), string='\n', start=(2, 14), end=(2, 15), line='def add(a, b):\n')
TokenInfo(type=5 (INDENT), string='    '

## Шаг 3: Синтаксический анализ — построение AST

In [3]:
import ast
import astpretty

parsed_ast = ast.parse(code)
astpretty.pprint(parsed_ast)

Module(
    body=[
        FunctionDef(
            lineno=2,
            col_offset=0,
            end_lineno=3,
            end_col_offset=16,
            name='add',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(lineno=2, col_offset=8, end_lineno=2, end_col_offset=9, arg='a', annotation=None, type_comment=None),
                    arg(lineno=2, col_offset=11, end_lineno=2, end_col_offset=12, arg='b', annotation=None, type_comment=None),
                ],
                vararg=None,
                kwonlyargs=[],
                kw_defaults=[],
                kwarg=None,
                defaults=[],
            ),
            body=[
                Return(
                    lineno=3,
                    col_offset=4,
                    end_lineno=3,
                    end_col_offset=16,
                    value=BinOp(
                        lineno=3,
                        col_offset=11,
                        e

## Шаг 4: Понимание структуры AST

**Объяснение структуры AST:**

- **Module**: Корневой узел, представляющий весь модуль (файл).
  - **body**: Список инструкций верхнего уровня.

- **FunctionDef**: Определение функции.
  - **name**: Имя функции (`'add'`).
  - **args**: Аргументы функции.
    - **args**: Список аргументов (`'a'`, `'b'`).
  - **body**: Список инструкций внутри функции.
    - **Return**: Инструкция возврата.
      - **value**: Возвращаемое значение.
        - **BinOp**: Бинарная операция.
          - **left**: Левая часть операции (`Name(id='a')`).
          - **op**: Оператор (`Add()`).
          - **right**: Правая часть операции (`Name(id='b')`).

- **Assign**: Инструкция присваивания.
  - **targets**: Список переменных, которым присваивается значение (`Name(id='result')`).
  - **value**: Выражение, результат которого присваивается.
    - **Call**: Вызов функции.
      - **func**: Функция, которую вызывают (`Name(id='add')`).
      - **args**: Аргументы вызова (`Constant(value=2)`, `Constant(value=3)`).

- **Expr**: Выражение (используется для вызова функций, результаты которых не присваиваются переменным).
  - **value**: Выражение, которое нужно вычислить.
    - **Call**: Вызов функции.
      - **func**: Функция (`Name(id='print')`).
      - **args**: Аргументы вызова (`Name(id='result')`).

---

## Шаг 5: Использование AST в интерпретаторе

**Как интерпретатор использует AST:**

1. **Анализ и оптимизация (если применимо)**: На основе AST интерпретатор может выполнять некоторые проверки и оптимизации кода.

2. **Компиляция в байткод**: AST преобразуется в байткод — набор инструкций, понятных виртуальной машине Python.

3. **Исполнение байткода**: Виртуальная машина Python (PVM) интерпретирует байткод и выполняет соответствующие операции.

---

## Шаг 6: Пример байткода

In [4]:
import dis

def add(a, b):
    return a + b

dis.dis(add)

  4           0 LOAD_FAST                0 (a)
              2 LOAD_FAST                1 (b)
              4 BINARY_ADD
              6 RETURN_VALUE


**Объяснение байткода:**

- **LOAD_FAST 0 (a)**: Загружает значение переменной `a` в стек.
- **LOAD_FAST 1 (b)**: Загружает значение переменной `b` в стек.
- **BINARY_ADD**: Извлекает два верхних значения из стека, складывает их и результат помещает обратно в стек.
- **RETURN_VALUE**: Возвращает верхнее значение из стека как результат функции.